In [1]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.utils import load_img, img_to_array

In [2]:
batch_size = 8
shape=100
# augmentation for training
train_datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in subfolders of the given directory, and generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'C:\\Users\\Albatol Ihab\\project\\train', 
        target_size=(shape, shape),
        batch_size=batch_size,
        class_mode='binary',
        subset='training') 

valid_generator = train_datagen.flow_from_directory(
    directory='C:\\Users\\Albatol Ihab\\project\\train',
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

test_generator = test_datagen.flow_from_directory(
        'C:\\Users\\Albatol Ihab\\project\\test',
        target_size=(shape, shape),
        batch_size=batch_size,
        class_mode=None)

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 20 images belonging to 1 classes.


In [3]:
#Building the CNN model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3),padding='same',activation='relu', input_shape=(shape, shape, 3)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D((2, 2)),   
  tf.keras.layers.Flatten(),  
  tf.keras.layers.Dense(20, activation='relu'),  
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [4]:
#Defining Loss function and optimizer
loss= tf.losses.binary_crossentropy
model.compile(loss=loss,optimizer="adam",metrics=['accuracy'])

In [5]:
#Training the model
history= model.fit(
        train_generator,
        epochs=15,
        steps_per_epoch = train_generator.n//train_generator.batch_size)

Epoch 1/15
10/10 [==============================] - 4s 91ms/step - loss: 0.7474 - accuracy: 0.5500
Epoch 2/15
10/10 [==============================] - 1s 92ms/step - loss: 0.7070 - accuracy: 0.6625
Epoch 3/15
10/10 [==============================] - 1s 90ms/step - loss: 0.6509 - accuracy: 0.6375
Epoch 4/15
10/10 [==============================] - 1s 94ms/step - loss: 0.5529 - accuracy: 0.6375
Epoch 5/15
10/10 [==============================] - 1s 90ms/step - loss: 0.4539 - accuracy: 0.8375
Epoch 6/15
10/10 [==============================] - 1s 88ms/step - loss: 0.3974 - accuracy: 0.9125
Epoch 7/15
10/10 [==============================] - 1s 91ms/step - loss: 0.4053 - accuracy: 0.9250
Epoch 8/15
10/10 [==============================] - 1s 88ms/step - loss: 0.4001 - accuracy: 0.9500
Epoch 9/15
10/10 [==============================] - 1s 87ms/step - loss: 0.3514 - accuracy: 0.9875
Epoch 10/15
10/10 [==============================] - 1s 110ms/step - loss: 0.3621 - accuracy: 0.9625
Epoch 11

In [ ]:
#model.save('1st_try.h5') 

In [ ]:
#from tensorflow.keras.models import load_model
#model.load_model('Albatol Ihab\\project\\1st_try.h5')

In [ ]:
# Make a plot for the loss
plt.xlabel('Epoch Number')
plt.ylabel("Loss Magnitude")
plt.plot(history.history['loss'])
plt.legend(['training'])

In [ ]:
# Make a plot for the accuracy
plt.xlabel('Epoch Number')
plt.ylabel("Accuracy")
plt.plot(history.history['accuracy'])
plt.legend(['training'])

In [ ]:
#Loading testing images for prediction and display it with its label
test_image = load_img('C:\\Users\\Albatol Ihab\\project\\test\\predict\\20230602_150203.jpg', target_size = (shape, shape))
plt.imshow(test_image)
plt.show()
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
if result <0.5:
    print('Apple')
elif result > 0.5:
    print('banana')
else: 
    print('none')

In [6]:
import os
import glob
import random

# Set the path to the directory containing the test images
test_dir = 'C:\\Users\\Albatol Ihab\\project\\test\\predict'

# Set the target image size
shape = 100

# Load the saved model
#model = tf.keras.models.load_model('C:\\Users\\Albatol Ihab\\project\\1st_try.h5')

# Get a list of all the image files in the test directory
image_files = glob.glob(os.path.join(test_dir, '*.jpg'))

# Shuffle the list of image files
random.shuffle(image_files)

# Create a list of the test images
test_images = []
for img_path in image_files:
    img = load_img(img_path, target_size=(shape, shape))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    test_images.append(img_array)

# Convert the list of test images to a numpy array
test_images = np.vstack(test_images)

# Normalize the test images
test_images = test_images / 255.0

# Use the model's predict() method to make predictions
predictions = model.predict(test_images)

# Display the predictions
for i, prediction in enumerate(predictions):
    if prediction > 0.5:
        print(f"Image {i+1}: Banana")
    else:
        print(f"Image {i+1}: Apple")

1/1 [==============================] - 0s 254ms/step
Image 1: Apple
Image 2: Apple
Image 3: Apple
Image 4: Apple
Image 5: Apple
Image 6: Apple
Image 7: Apple
Image 8: Apple
Image 9: Apple
Image 10: Apple
Image 11: Banana
Image 12: Banana
Image 13: Banana
Image 14: Banana
Image 15: Banana
Image 16: Banana
Image 17: Banana
Image 18: Banana
Image 19: Banana
Image 20: Banana


In [10]:
#Real-time Detection
# Open the camera
vid = cv2.VideoCapture(0)
print("Camera connection successfully established")

while(True):  
    # Capture the frame from camera
    r, frame = vid.read() 
    cv2.imshow('frame', frame)
    
    # Save the frame to disk
    cv2.imwrite('C:\\Users\\Albatol Ihab\\project\\preview.jpg', frame)
    
    # Load the saved image and preprocess it
    test_image = load_img('C:\\Users\\Albatol Ihab\\project\\preview.jpg', target_size = (shape, shape))
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    
    # Get the predicted label and draw a bounding box around the object
    label = ""
    if result[0][0] >0.5:
        print("I saw a banana")
        label = "banana"
        color = (0, 255, 0)  # green color for banana
    elif result[0][0] < 0.5:
        print("I saw an apple")
        label = "apple"
        color = (0, 0, 255)  # red color for apple
    else:
        label = "unknown"
        color = (255, 0, 0)  # blue color for unknown object
        
    # Get the coordinates of the object in the image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(gray, 50, 150)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw the bounding box and label on the frame
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w > 20 and h > 20:
            cv2.putText(frame, label,(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            break
    
    cv2.imshow('frame', frame)
    
    # Remove the saved image from disk
    os.remove('C:\\Users\\Albatol Ihab\\project\\preview.jpg')
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

# Release the camera and close all windows
vid.release() 
cv2.destroyAllWindows()

Camera connection successfully established
1/1 [==============================] - 0s 54ms/step
I saw a banana
1/1 [==============================] - 0s 63ms/step
I saw a banana
1/1 [==============================] - 0s 51ms/step
I saw a banana
1/1 [==============================] - 0s 61ms/step
I saw a banana
1/1 [==============================] - 0s 57ms/step
I saw a banana
1/1 [==============================] - 0s 69ms/step
I saw a banana
1/1 [==============================] - 0s 48ms/step
I saw a banana
1/1 [==============================] - 0s 64ms/step
I saw a banana
1/1 [==============================] - 0s 62ms/step
I saw a banana
1/1 [==============================] - 0s 75ms/step
I saw a banana
1/1 [==============================] - 0s 55ms/step
I saw a banana
1/1 [==============================] - 0s 55ms/step
I saw a banana
1/1 [==============================] - 0s 67ms/step
I saw a banana
1/1 [==============================] - 0s 60ms/step
I saw a banana
1/1 [==============

KeyboardInterrupt: 